In [36]:
import sys
import dataclasses
import pandas as pd
import numpy as np
import logging
sys.path.append('..')

In [37]:
from rnacloud_genome_reference.common.gtf import Feature, Exon, Intron, GTFHandler
from rnacloud_genome_reference.common.gnomad import GenomicRegionQuerier, query_genomic_regions

In [38]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [39]:
a = GTFHandler('../data/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.sorted.gtf.gz')

In [40]:
protein_coding_genes = pd.read_csv("../temp/protein_coding_genes.tsv", sep="\t", low_memory=False)

In [41]:
protein_coding_genes.head()

,chr,start,end,strand,gene_name,entrez_gene_id
0,NC_000001.11,65419,71585,+,OR4F5,79501
1,NC_000001.11,365134,382235,-,LOC112268260,112268260
2,NC_000001.11,450740,451678,-,OR4F29,729759
3,NC_000001.11,586287,611297,-,LOC105378947,105378947
4,NC_000001.11,676076,720101,-,OR4F16,81399


In [42]:
clinical_genes = pd.read_csv("../data/clinically_relevant_genes/1.0.2/results.tsv", sep="\t", low_memory=False)

In [43]:
clinical_genes.columns

Index(['hgnc_id', 'symbol', 'name', 'locus_group', 'locus_type', 'status',
       'location', 'location_sortable', 'alias_symbol', 'alias_name',
       'prev_symbol', 'prev_name', 'gene_group', 'gene_group_id',
       'date_approved_reserved', 'date_symbol_changed', 'date_name_changed',
       'date_modified', 'entrez_id', 'ensembl_gene_id', 'vega_id', 'ucsc_id',
       'ena', 'refseq_accession', 'ccds_id', 'uniprot_ids', 'pubmed_id',
       'in_omim', 'in_panelapp_uk'],
      dtype='object')

In [44]:
clinical_genes.locus_group.value_counts()

locus_group
protein-coding gene    4536
non-coding RNA           42
other                     7
pseudogene                1
Name: count, dtype: int64

In [45]:
protein_coding_genes['clinically_relevant'] = np.where(
    protein_coding_genes['entrez_gene_id'].isin(clinical_genes.loc[clinical_genes.locus_group == "protein-coding gene", 'entrez_id']),
    True,
    False
)

In [60]:
genome_regions = pd.read_csv('../data/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_assembly_report.txt', 
                      sep='\t',
                      comment='#',
                      low_memory=False,
                      header=None,
                      names=['Sequence-Name','Sequence-Role','Assigned-Molecule','Assigned-Molecule-Location/Type','GenBank-Accn','Relationship','RefSeq-Accn','Assembly-Unit','Sequence-Length','UCSC-style-name'])

In [68]:
genome_regions.query('`Sequence-Role`=="assembled-molecule"')

,Sequence-Name,Sequence-Role,Assigned-Molecule,Assigned-Molecule-Location/Type,GenBank-Accn,Relationship,RefSeq-Accn,Assembly-Unit,Sequence-Length,UCSC-style-name
0,1,assembled-molecule,1,Chromosome,CM000663.2,=,NC_000001.11,Primary Assembly,248956422,chr1
1,2,assembled-molecule,2,Chromosome,CM000664.2,=,NC_000002.12,Primary Assembly,242193529,chr2
2,3,assembled-molecule,3,Chromosome,CM000665.2,=,NC_000003.12,Primary Assembly,198295559,chr3
3,4,assembled-molecule,4,Chromosome,CM000666.2,=,NC_000004.12,Primary Assembly,190214555,chr4
4,5,assembled-molecule,5,Chromosome,CM000667.2,=,NC_000005.10,Primary Assembly,181538259,chr5
5,6,assembled-molecule,6,Chromosome,CM000668.2,=,NC_000006.12,Primary Assembly,170805979,chr6
6,7,assembled-molecule,7,Chromosome,CM000669.2,=,NC_000007.14,Primary Assembly,159345973,chr7
7,8,assembled-molecule,8,Chromosome,CM000670.2,=,NC_000008.11,Primary Assembly,145138636,chr8
8,9,assembled-molecule,9,Chromosome,CM000671.2,=,NC_000009.12,Primary Assembly,138394717,chr9
9,10,assembled-molecule,10,Chromosome,CM000672.2,=,NC_000010.11,Primary Assembly,133797422,chr10


In [71]:
protein_coding_genes = protein_coding_genes.merge(
    genome_regions[['Sequence-Name','RefSeq-Accn','Sequence-Role']].rename(columns={'Sequence-Name': 'chrom', 
                                                                                'RefSeq-Accn': 'chr', 
                                                                                'Sequence-Role': 'role'}),
    how='left',
    on='chr'
)

In [76]:
pcg_of_interest = protein_coding_genes.query('role == "assembled-molecule" and chrom != "MT" and clinically_relevant == True')

,chrom,start,end,gene_name,entrez_gene_id
11,1,1013497,1014540,ISG15,9636
12,1,1020120,1056116,AGRN,375790
19,1,1232237,1235041,B3GALT6,126792
25,1,1311600,1324660,INTS11,54973
28,1,1335278,1349418,DVL1,1855
...,...,...,...,...,...
19846,Y,624344,659411,SHOX,6473
19848,Y,1268814,1325218,CSF2RA,1438
19856,Y,2219506,2500976,DHRSX,207063
19859,Y,2786855,2787682,SRY,6736


In [62]:
protein_coding_genes

,chr,start,end,strand,gene_name,entrez_gene_id,clinically_relevant
0,NC_000001.11,65419,71585,+,OR4F5,79501,False
1,NC_000001.11,365134,382235,-,LOC112268260,112268260,False
2,NC_000001.11,450740,451678,-,OR4F29,729759,False
3,NC_000001.11,586287,611297,-,LOC105378947,105378947,False
4,NC_000001.11,676076,720101,-,OR4F16,81399,False
...,...,...,...,...,...,...,...
23307,NC_012920.1,10470,10766,+,ND4L,4539,True
23308,NC_012920.1,10760,12137,+,ND4,4538,True
23309,NC_012920.1,12337,14148,+,ND5,4540,True
23310,NC_012920.1,14149,14673,-,ND6,4541,True


In [ ]:
genome_regions[]

,Sequence-Name,Sequence-Role,Assigned-Molecule,Assigned-Molecule-Location/Type,GenBank-Accn,Relationship,RefSeq-Accn,Assembly-Unit,Sequence-Length,UCSC-style-name
0,1,assembled-molecule,1,Chromosome,CM000663.2,=,NC_000001.11,Primary Assembly,248956422,chr1
1,2,assembled-molecule,2,Chromosome,CM000664.2,=,NC_000002.12,Primary Assembly,242193529,chr2
2,3,assembled-molecule,3,Chromosome,CM000665.2,=,NC_000003.12,Primary Assembly,198295559,chr3
3,4,assembled-molecule,4,Chromosome,CM000666.2,=,NC_000004.12,Primary Assembly,190214555,chr4
4,5,assembled-molecule,5,Chromosome,CM000667.2,=,NC_000005.10,Primary Assembly,181538259,chr5
...,...,...,...,...,...,...,...,...,...,...
704,HSCHR19KIR_FH13_A_HAP_CTG3_1,alt-scaffold,19,Chromosome,KI270931.1,=,NT_187685.1,ALT_REF_LOCI_32,170148,chr19_KI270931v1_alt
705,HSCHR19KIR_FH13_BA2_HAP_CTG3_1,alt-scaffold,19,Chromosome,KI270932.1,=,NT_187686.1,ALT_REF_LOCI_33,215732,chr19_KI270932v1_alt
706,HSCHR19KIR_FH15_A_HAP_CTG3_1,alt-scaffold,19,Chromosome,KI270933.1,=,NT_187687.1,ALT_REF_LOCI_34,170537,chr19_KI270933v1_alt
707,HSCHR19KIR_RP5_B_HAP_CTG3_1,alt-scaffold,19,Chromosome,GL000209.2,=,NT_113949.2,ALT_REF_LOCI_35,177381,chr19_GL000209v2_alt


In [46]:
protein_coding_genes[protein_coding_genes['clinically_relevant']]

,chr,start,end,strand,gene_name,entrez_gene_id,clinically_relevant
11,NC_000001.11,1013497,1014540,+,ISG15,9636,True
12,NC_000001.11,1020120,1056116,+,AGRN,375790,True
19,NC_000001.11,1232237,1235041,+,B3GALT6,126792,True
25,NC_000001.11,1311600,1324660,-,INTS11,54973,True
28,NC_000001.11,1335278,1349418,-,DVL1,1855,True
...,...,...,...,...,...,...,...
23307,NC_012920.1,10470,10766,+,ND4L,4539,True
23308,NC_012920.1,10760,12137,+,ND4,4538,True
23309,NC_012920.1,12337,14148,+,ND5,4540,True
23310,NC_012920.1,14149,14673,-,ND6,4541,True


In [47]:
a.get_transcript_for_gene('NC_000001.11', 1013497, 1014540, 9636, True)

INFO:rnacloud_genome_reference.common.gtf:Obtaining transcript for Entrez Gene ID: 9636 at location NC_000001.11:1013497-1014540 (MANE: True)


'NM_005101.4'

In [48]:
x = a.get_exons_by_transcript('NC_000001.11', 1013497, 1014540, 'NM_005101.4')

INFO:rnacloud_genome_reference.common.gtf:Obtaining exons for transcript: NM_005101.4 at location NC_000001.11:1013497-1014540


In [54]:
x

[Exon(chromosome='NC_000001.11', start=1013497, end=1013576, strand='+', sequence=None, exon_no=1),
 Exon(chromosome='NC_000001.11', start=1013984, end=1014540, strand='+', sequence=None, exon_no=2)]

In [ ]:
if len(x) > 0:
    for entry, exon in enumerate(x):
        if exon.strand == '+':
            if entry == 0:


0
1


In [50]:
example_regions = [
    {"chrom": "1", "start": 1013497, "stop": 1013497}
]

In [51]:
endpoint = "https://gnomad.broadinstitute.org/api"

In [52]:
query_genomic_regions(example_regions, endpoint)

INFO:rnacloud_genome_reference.common.gnomad:Initialized GenomicRegionQuerier with endpoint: https://gnomad.broadinstitute.org/api
INFO:rnacloud_genome_reference.common.gnomad:Querying 1 genomic regions
INFO:gql.transport.requests:>>> {"query": "query MultipleRegions {\n  region1: region(\n    chrom: \"1\"\n    start: 1013497\n    stop: 1013497\n    reference_genome: GRCh38\n  ) {\n    variants(dataset: gnomad_r4) {\n      chrom\n      pos\n      alt\n      lof_filter\n      joint {\n        ac\n        an\n        hemizygote_count\n        homozygote_count\n        filters\n      }\n    }\n  }\n}"}
INFO:gql.transport.requests:<<< {"data":{"region1":{"variants":[{"chrom":"1","pos":1013497,"alt":"C","lof_filter":null,"joint":{"ac":1,"an":1486702,"hemizygote_count":0,"homozygote_count":0,"filters":[]}},{"chrom":"1","pos":1013497,"alt":"T","lof_filter":null,"joint":{"ac":1,"an":1486700,"hemizygote_count":0,"homozygote_count":0,"filters":[]}},{"chrom":"1","pos":1013497,"alt":"A","lof_filte

[{'region': {'chrom': '1', 'start': 1013497, 'stop': 1013497},
  'variants': [{'chrom': '1',
    'pos': 1013497,
    'alt': 'C',
    'lof_filter': None,
    'joint': {'ac': 1,
     'an': 1486702,
     'hemizygote_count': 0,
     'homozygote_count': 0,
     'filters': []}},
   {'chrom': '1',
    'pos': 1013497,
    'alt': 'T',
    'lof_filter': None,
    'joint': {'ac': 1,
     'an': 1486700,
     'hemizygote_count': 0,
     'homozygote_count': 0,
     'filters': []}},
   {'chrom': '1',
    'pos': 1013497,
    'alt': 'A',
    'lof_filter': None,
    'joint': {'ac': 18,
     'an': 1486702,
     'hemizygote_count': 0,
     'homozygote_count': 0,
     'filters': []}}]}]